In [1]:
import sqlite3
import pandas as pd

In [2]:
db = 'chinook.db'

In [3]:
def run_query(query):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(query, conn)

In [4]:
def run_command(command):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(command)

In [5]:
def show_tables():
    query = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table", "view");
    '''
    return run_query(query)

show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


In [6]:
genre_sold_usa = '''
WITH usa_sold AS
    (
     SELECT il.*
     FROM invoice_line il
     INNER JOIN invoice i ON il.invoice_id = i.invoice_id
     INNER JOIN customer c ON i.customer_id = c.customer_id
     WHERE c.country = 'USA'
     )
     
SELECT
    g.name genre,
    COUNT(us.invoice_line_id) tracks_sold,
    CAST(COUNT(us.invoice_line_id) AS FLOAT) / 
    (SELECT COUNT(*) FROM usa_sold) percentage_sold
FROM usa_sold us
INNER JOIN track t ON us.track_id = t.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY genre
ORDER BY percentage_sold DESC
'''

run_query(genre_sold_usa)

,genre,tracks_sold,percentage_sold
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


The population for this data is all the tracks sold in the USA.

The three columns are broken down as:
    - genre: The genre of music
    - tracks_sold: The number of tracks sold in the population
    - percentage_sold: The percentage of tracks sold within the population
    
The artists whose albums we should purchase from the store are:
- Red Tone: Punk
- Slim Jim Bites: Blues
- Meteor and the Girls: Pop

because these genres are more popular purchases at the store than Hip-Hop done by Regal.

In [7]:
total_sales_agent = '''
WITH sales AS
    (
     SELECT 
         i.customer_id,
         c.support_rep_id,
         SUM(i.total) total
     FROM invoice i
     INNER JOIN customer c ON i.customer_id = c.customer_id
     GROUP BY c.customer_id, c.support_rep_id
     )
     
SELECT 
    e.employee_id,
    e.first_name || " " || e.last_name full_name,
    SUM(s.total) total_sales
    
FROM employee e
INNER JOIN sales s ON e.employee_id = s.support_rep_id
GROUP BY e.employee_id
ORDER BY total_sales DESC
'''

run_query(total_sales_agent)

,employee_id,full_name,total_sales
0,3,Jane Peacock,1731.51
1,4,Margaret Park,1584.00
2,5,Steve Johnson,1393.92


The total sales for each agent is displayed on the above table. Jane Peacock appears to have the most sales in dollars out of all employees

In [8]:
sales_by_country = '''
WITH country_other AS
    (
     SELECT
         CASE
             WHEN (
                  SELECT count(*)
                  FROM customer
                  WHERE country = c.country
                  ) = 1 THEN "Other"
                  ELSE c.country
                  END AS country,
            c.customer_id,
            il.*
        FROM invoice_line il
        INNER JOIN invoice i ON i.invoice_id = il.invoice_id
        INNER JOIN customer c ON c.customer_id = i.customer_id
      )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        COUNT(DISTINCT customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / COUNT(DISTINCT customer_id) customer_lifetime_value,
        SUM(unit_price) / COUNT(DISTINCT invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
            END AS order_other_bottom
        FROM country_other
        GROUP BY country
        ORDER BY order_other_bottom, total_sales DESC
);
'''

run_query(sales_by_country)

,country,customers,total_sales,average_order,customer_lifetime_value
0,USA,13,1040.49,7.942672,80.037692
1,Canada,8,535.59,7.047237,66.948750
2,Brazil,5,427.68,7.011148,85.536000
3,France,5,389.07,7.781400,77.814000
4,Germany,4,334.62,8.161463,83.655000
5,Czech Republic,2,273.24,9.108000,136.620000
6,United Kingdom,3,245.52,8.768571,81.840000
7,Portugal,2,185.13,6.383793,92.565000
8,India,2,183.15,8.721429,91.575000
9,Other,15,1094.94,7.448571,72.996000


USA is the country that has the most customers with 13 and most sales with 1040.49. There is high growth potential in sales in Czech Republic as the average order is the highest of every country. If there can be more customers while mainting the average order, the total sales amount would increase the fastest.

In [17]:
album_or_not = '''
WITH invoice_first_track AS
    (
     SELECT
         invoice_id,
         MIN(track_id) first_track_id
         FROM invoice_line
         GROUP BY invoice_id
    )
    
SELECT
    album_purchase,
    COUNT(invoice_id) amount_of_invoice,
    CAST(COUNT(invoice_id) AS FLOAT) / (
                                        SELECT COUNT(*)
                                        FROM invoice
                                        ) percent
FROM
    (
     SELECT
         ifs.*,
         CASE
             WHEN
             (
              SELECT t.track_id 
              FROM track t
              WHERE t.album_id = (
                                  SELECT t2.album_id
                                  FROM track t2
                                  WHERE t2.track_id = ifs.first_track_id
                                  )
              EXCEPT 
              SELECT il2.track_id 
              FROM invoice_line il2
              WHERE il2.invoice_id = invoice_id
              ) IS NULL
            AND
             (
              SELECT il2.track_id FROM invoice_line il2
              WHERE il2.invoice_id = ifs.invoice_id
              EXCEPT
              SELECT t.track_id
              FROM track t
              WHERE t.album_id = (
                                  SELECT t2.album_id
                                  FROM track t2
                                  WHERE t2.track_id = ifs.first_track_id
                                  )
              ) IS NULL
           THEN "yes"
           ELSE "no"
        END AS album_purchase
    FROM invoice_first_track ifs
    )
GROUP BY album_purchase
'''

run_query(album_or_not)

,album_purchase,amount_of_invoice,percent
0,no,454,0.739414
1,yes,160,0.260586


Albums account for about 26% of purchasees. Since over a quarter of sales are made from albums, the Chinook store should continue to buy full albums from record companies.